#### Import libraries

In [ ]:
import time
import numpy as np
import pandas as pd

import os
from os.path import join

import pynq
from pynq import Overlay

import functions as ft
from v2 import helper as hp

#### Load overlay

In [ ]:
print("Loading Overlay and setting up AXI Regs IP...")

ol = Overlay("/home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/v2/hwh_and_bit/design_1_wrapper.bit")
axi_regs_ip = ol.axi_regs_v2_0

print(f"Overlay loaded successfully.")

Loading Overlay and setting up AXI Regs IP...


Overlay loaded successfully.


#### Set initial state to AXI Registers

In [3]:
print(f"Found IP: axi_regs_v2_0 at base address {hex(axi_regs_ip.mmio.base_addr)}")

# AXI Regs v2 IP ================================================================================ #
#
# 0x00 - 0x11	:	lfsr ros counters		-	[R] for PS, [W] by ring_oscilaltor_puf_v2
# 0x12 - 0x31	:	main ros counters		-	[R] for PS, [W] by ring_oscilaltor_puf_v2
# 0x32 - 0x35	:	puf response			-	[R] for PS, [W] by ring_oscilaltor_puf_v2
# 0x36			:	lfsr ros corrected		-	[R/W] for PS, [R] by ring_oscilaltor_puf_v2
# 0x37 - 0x3A	:	puf response corrected	-	[R/W] for PS
#
# 0x3B			:	control reg
#
#   - Bit 0     :   reset_n                 -   [R/W]
#   - Bit 1     :   enable                  -   [R/W]
#   - Bit 2     :   lfsr ros counters ready -   [R]
#   - Bit 3     :   main ros counters ready -   [R]
#   - Bit 4     :   puf response ready      -   [R]
#
# =============================================================================================== #

print("Setting initial state...")
# To disable enable and reset_n -> Set to "00" control signals register LSBs (register 0x3B) -> 0x3B * 4 = 0xEC
print(f"Writing value 0 to control register (slv_reg59)...")
axi_regs_ip.write(hp.CONTROL_REG_ADDR, 0)
print("Hardware initialization complete.")

# Read back the value to confirm the write operation
read_value = axi_regs_ip.read(hp.CONTROL_REG_ADDR)
print(f"Read back value: {hex(read_value)}")

Found IP: axi_regs_v2_0 at base address 0x43c00000
Setting initial state...
Writing value 0 to control register (slv_reg59)...
Hardware initialization complete.
Read back value: 0x0


#### Get PUF data

In [ ]:
# --- Step 1: Apply System Reset ---
axi_regs_ip.write(hp.CONTROL_REG_ADDR, 0x00000000)
time.sleep(0.01)
axi_regs_ip.write(hp.CONTROL_REG_ADDR, 0x00000001) ; # Enable low (disabled), reset_n high (disabled)
#print("Released reset signal.")

# --- Phase 1: LFSR ROs Counter Reading ---
# Wait for the LFSR RO counters to indicate they are ready with sampled values
#print("Waiting for LFSR RO counters to be ready...")
while (axi_regs_ip.read(hp.CONTROL_REG_ADDR) & hp.LFSR_ROS_COUNTERS_READY_BIT_MASK) == 0:
    time.sleep(0.001)
# Clear the flag
current_control_val = axi_regs_ip.read(hp.CONTROL_REG_ADDR)
axi_regs_ip.write(hp.CONTROL_REG_ADDR, current_control_val & ~hp.LFSR_ROS_COUNTERS_READY_BIT_MASK)

# Give a moment for signals to settle after ready assertion
time.sleep(0.001)

lfsr_ros_counters = []

# Fetch lfsr seed AXI reg values, and get the temperature and vccint at that moment (before and after -> mean)
lfsr_seed_vccint1 = hp.get_zynq_vccint()
lfsr_seed_temperature1 = hp.get_zynq_temperature()
for i in range(hp.LFSR_REG_COUNT):
    offset = hp.LFSR_ROS_COUNTERS_ADDR + (i * 4)
    value = axi_regs_ip.read(offset)
    lfsr_ros_counters.append(value)
lfsr_seed_vccint2 = hp.get_zynq_vccint()
lfsr_seed_temperature2 = hp.get_zynq_temperature()

# Compute the mean of both measurements for vccint and temperature
lfsr_seed_vccint = (lfsr_seed_vccint1 + lfsr_seed_vccint2) / 2
lfsr_seed_temperature = (lfsr_seed_temperature1 + lfsr_seed_temperature2) / 2

# print("Counters data:")
# for value in lfsr_ros_counters:
#     print(value)

# Process LFSR RO counters into seed bits (based on comparison of pairs)
seed_bits = []
for i in range(0, len(lfsr_ros_counters) - 1, 2):
    val_first = lfsr_ros_counters[i]
    val_second = lfsr_ros_counters[i+1]

    if val_second < val_first:
        seed_bits.append('1')
    else:
        seed_bits.append('0')

seed_bits.reverse()
lfsr_seed_value = "".join(seed_bits)

# print(f"\nLFSR seed value: 0b{lfsr_seed_value}")

# Send the corrected value for lfsr seed. The axi_regs_v2 ip sends an ack to ring_oscillator_puf_v2
lfsr_seed_int = int(lfsr_seed_value, 2)
axi_regs_ip.write(hp.LFSR_ROS_CORRECTED_ADDR, lfsr_seed_int) ; #!
#print("LFSR counters ready signal acknowledged.")

# --- Phase 2: Main ROs Counter Reading ---
# Enable the main ROs and counters to start accumulation for the PUF response
#print("Enabling main ROs and counters...")
axi_regs_ip.write(hp.CONTROL_REG_ADDR, 0x00000003) ; # Enable active, reset_n inactive

# Wait for the main RO counters to indicate they are ready with sampled values
#print("Waiting for main RO counters to be ready...")
while (axi_regs_ip.read(hp.CONTROL_REG_ADDR) & hp.MAIN_ROS_COUNTERS_READY_BIT_MASK) == 0:
    time.sleep(0.01)
# Clear the flag
current_control_val = axi_regs_ip.read(hp.CONTROL_REG_ADDR)
axi_regs_ip.write(hp.CONTROL_REG_ADDR, current_control_val & ~hp.MAIN_ROS_COUNTERS_READY_BIT_MASK)

main_ros_counters = []

# Fetch main AXI reg values, and get the temperature at that moment
puf_response_vccint1 = hp.get_zynq_vccint()
puf_response_temperature1 = hp.get_zynq_temperature()
for i in range(hp.MAIN_REG_COUNT):
    offset = hp.MAIN_ROS_COUNTERS_ADDR + (i * 4)
    value = axi_regs_ip.read(offset)
    main_ros_counters.append(value)
puf_response_vccint2 = hp.get_zynq_vccint()
puf_response_temperature2 = hp.get_zynq_temperature()

# Compute the mean of both measurements for vccint and temperature
puf_response_vccint = (puf_response_vccint1 + puf_response_vccint2) / 2
puf_response_temperature = (puf_response_temperature1 + puf_response_temperature2) / 2

# Acknowledge to ring_oscillator_puf_v2 is sent as soon as the axi_regs_ip_v2 registers the main ros counters

# --- Phase 3: Final 128-bit PUF Response Reading ---
# Wait for the final 128-bit PUF response to be ready from the hardware
#print("Waiting for final PUF response to be ready...")
while (axi_regs_ip.read(hp.CONTROL_REG_ADDR) & hp.PUF_RESPONSE_READY_BIT_MASK) == 0:
    time.sleep(0.01)
# Clear the flag, and disable the main enable once the final response is ready
axi_regs_ip.write(hp.CONTROL_REG_ADDR, 0x00000001) ; # Enable inactive, reset_n inactive

# Read the final 128-bit PUF response
puf_response = []
for i in range(hp.PUF_RESPONSE_REG_COUNT):
    offset = hp.PUF_RESPONSE_ADDR + i * 4
    puf_response.append(axi_regs_ip.read(offset))

# Convert each 32-bit integer to a 32-digit binary string and join them
puf_response_value = "".join(f'{val:032b}' for val in reversed(puf_response))

# print(f"128-bit PUF Response: 0b{puf_response_value}")

# Same as with main ros counters, acknowledge to ring_oscillator_puf_v2 is sent as soon as the axi_regs_ip_v2 registers the puf response values

#print("Final PUF response ready signal acknowledged.")

puf_response_data = (puf_response_vccint, puf_response_temperature, puf_response_value)

In [ ]:
print("\nReleasing GPIO resources and resetting hardware.")
# Ensure all output GPIOs are returned to a known safe state
axi_regs_ip.write(hp.CONTROL_REG_ADDR, 0)

# Release the overlay to free up hardware resources
# This implicitly releases GPIOs and AXI drivers
# ol.free() 
print("Hardware cleanup complete.")


Releasing GPIO resources and resetting hardware.
Hardware cleanup complete.


#### Correct response

#### Loading PUF Response MLP Agent weights and biases

In [6]:
puf_response_mlp_agent_weights_and_biases = r'/home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/puf_response_mlp_agent_weights_and_biases'
print(f"Loading weights and biases from: {puf_response_mlp_agent_weights_and_biases}")

# Load weights from .npy files
w1 = np.load(join(puf_response_mlp_agent_weights_and_biases, 'puf_response_mlp_agent_w1.npy'))
w2 = np.load(join(puf_response_mlp_agent_weights_and_biases, 'puf_response_mlp_agent_w2.npy'))
w_out = np.load(join(puf_response_mlp_agent_weights_and_biases, 'puf_response_mlp_agent_w3.npy'))

# Load biases from .npy files
b1 = np.load(join(puf_response_mlp_agent_weights_and_biases, 'puf_response_mlp_agent_b1.npy'))
b2 = np.load(join(puf_response_mlp_agent_weights_and_biases, 'puf_response_mlp_agent_b2.npy'))
b_out = np.load(join(puf_response_mlp_agent_weights_and_biases, 'puf_response_mlp_agent_b3.npy'))

# Ensure biases are 2D row vectors for broadcasting in the forward pass
# Keras saves biases as 1D (N,), but NumPy math expects (1, N)
b1 = b1.reshape(1, -1)
b2 = b2.reshape(1, -1)
b_out = b_out.reshape(1, -1)

Loading weights and biases from: /home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/puf_response_mlp_agent_weights_and_biases


#### Correct PUF response

In [ ]:
print(f"VccInt: {puf_response_data[0]}")
print(f"Temperature: {puf_response_data[1]}")
print(f"Response: {puf_response_data[2]}")
response_before = hex(int(puf_response_data[2], 2))
print(f"Response (hex): {response_before}")

VccInt: 1.0235595703125
Temperature: 54.876696777343795
Response: 10111011010001000010001000101010000010100011011010011001001111110110011100010010000001000010100001000111100110001100100001010001


Right now VccInt and Temperature values are not normalized. To do so, the script needs the VccInt and Temperature dataset for gettin min and max ranges, and getting a value for both between 0 and 1.

In [ ]:
pynq_1_raw_data_path = r'/home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/raw_data/pynq_1_data.csv'
df = pd.read_csv(pynq_1_raw_data_path, dtype=str)

section = 'PUF_Response'

df[f'{section}_Vccint'] = pd.to_numeric(df[f'{section}_Vccint'], errors='coerce')
df[f'{section}_Temperature'] = pd.to_numeric(df[f'{section}_Temperature'], errors='coerce')

vccint_normalized = (puf_response_data[0] - df[f'{section}_Vccint'].min()) / (df[f'{section}_Vccint'].max() - df[f'{section}_Vccint'].min())
temperature_normalized = (puf_response_data[1] - df[f'{section}_Temperature'].min()) / (df[f'{section}_Temperature'].max() - df[f'{section}_Temperature'].min())

# Convert the 128-bit string into a NumPy array of integers (0s and 1s)
bit_string = puf_response_data[2]
bit_array = np.array(list(bit_string), dtype=int)

# Concatenate the two normalized values with the bit array
input_vector = np.concatenate([
    [vccint_normalized], 
    [temperature_normalized], 
    bit_array
])

# Verify the result
print("Shape of the final vector:", input_vector.shape)
print("First 5 elements:", input_vector[:5])
print("Last 5 elements:", input_vector[-5:])

Shape of the final vector: (130,)
First 5 elements: [0.61538462 0.72534143 1.         0.         1.        ]
Last 5 elements: [1. 0. 0. 0. 1.]


In [9]:
unlock_key = ft.forward_pass_puf_response(input_vector, w1, b1, w2, b2, w_out, b_out)

In [10]:
print("Unlock key after agent: ", unlock_key)

Unlock key after agent:  [[ 18.01626389 -19.21937694  18.72154119  19.50730031  17.7319477
  -18.80423446  18.11436111  17.54001106 -18.09532394  17.10485545
  -18.0910915  -17.36096579 -19.41690913  18.10141678 -20.26380517
  -19.16645418 -20.67561026 -19.40113726  19.43355224 -19.32157277
  -16.29813002 -18.23382162  19.73082725 -20.7402903  -19.66368667
  -19.36623633  19.29189496 -17.76140926  18.34891762 -18.03606782
   18.08992839 -17.54399191 -18.79097073 -17.98284732 -19.21643237
  -19.81251487  16.30376307 -18.1261448   19.00995929 -19.33764088
  -18.41149996 -18.32838911  17.06293356  17.15167973 -16.70079309
   18.6154985   18.35038812 -18.37995167  18.69185794 -19.00984547
  -19.07039941  16.42129643  16.87642349 -19.16046686 -19.28357916
   17.65190318 -18.35390656 -19.41298107  16.74670201  18.10470276
   16.09383119  16.23506084  19.46921607  17.35166045 -18.15311462
   19.80964006  18.34032269 -19.23163132 -18.13414209  19.09617536
   18.10426127  17.26736191 -19.613912

In [ ]:
# Flatten the key from (1, 128) to (128,)
key_flattened = unlock_key.flatten()

# Binarize the logits using a fixed threshold of 0
binarized_bits = (key_flattened > 0).astype(int)

print("First 16 binarized bits:", binarized_bits[:16])

bit_groups = binarized_bits.reshape(8, 16)

# Convert each 16-bit group into a single integer
unlock_key_processed = []
for group in bit_groups:
    # Join the bits ([1, 0, 1, ...]) into a binary string ("101...")
    bit_string = "".join(group.astype(str))
    # Convert the binary string into its base-10 integer equivalent
    integer_value = int(bit_string, 2)
    unlock_key_processed.append(integer_value)

print("Processed Key (list of 8 integers):")
print(unlock_key_processed)
print(f"\nLength of processed key: {len(unlock_key_processed)}")

First 16 binarized bits: [1 0 1 1 1 0 1 1 0 1 0 0 0 1 0 0]
Processed Key (list of 8 integers):
[47940, 8746, 2614, 39231, 26386, 1064, 18328, 51281]

Length of processed key: 8


#### Loading locked MNIST weights and biases

In [ ]:
mnist_mlp_weights_and_biases_locked_path = r'/home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/mnist_mlp_weights_and_biases_locked'
print(f"Loading weights and biases from: {mnist_mlp_weights_and_biases_locked_path}")

# Load weights from .npy files
w1 = np.load(join(mnist_mlp_weights_and_biases_locked_path, 'mnist_mlp_w1.npy'))
w2 = np.load(join(mnist_mlp_weights_and_biases_locked_path, 'mnist_mlp_w2.npy'))
w3 = np.load(join(mnist_mlp_weights_and_biases_locked_path, 'mnist_mlp_w3.npy'))
w_out = np.load(join(mnist_mlp_weights_and_biases_locked_path, 'mnist_mlp_w4.npy'))

# Load biases from .npy files
b1 = np.load(join(mnist_mlp_weights_and_biases_locked_path, 'mnist_mlp_b1.npy'))
b2 = np.load(join(mnist_mlp_weights_and_biases_locked_path, 'mnist_mlp_b2.npy'))
b3 = np.load(join(mnist_mlp_weights_and_biases_locked_path, 'mnist_mlp_b3.npy'))
b_out = np.load(join(mnist_mlp_weights_and_biases_locked_path, 'mnist_mlp_b4.npy'))

# Ensure biases are 2D row vectors for broadcasting in the forward pass
# Keras saves biases as 1D (N,), but NumPy math expects (1, N)
b1 = b1.reshape(1, -1)
b2 = b2.reshape(1, -1)
b3 = b3.reshape(1, -1)
b_out = b_out.reshape(1, -1)

Loading weights and biases from: /home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/mnist_mlp_weights_and_biases_locked
PYNQ b1 mean: 0.0070, std: 0.0342


#### Load MNIST test dataset

In [ ]:
print("\nLoading MNIST test data from binary files...")
_training_images_filepath = ""
_training_labels_filepath = ""
_test_images_filepath = r'/home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/mnist_test_data/t10k-images.idx3-ubyte'
_test_labels_filepath = r'/home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/mnist_test_data/t10k-labels.idx1-ubyte'

mnist_data_loader = ft.MNISTDataLoader(
    _training_images_filepath, _training_labels_filepath,
    _test_images_filepath, _test_labels_filepath
)
x_test = mnist_data_loader.x_test
y_test_one_hot = mnist_data_loader.y_test # Store one-hot encoded labels
    
# For accuracy calculation, we need the integer labels, not one-hot
y_test = np.argmax(y_test_one_hot, axis=1) 

# print(f"Data check -> Min: {x_test.min()}, Max: {x_test.max()}")
    
print("MNIST test data loaded and preprocessed.")


Loading MNIST test data from binary files...
MNIST Test Data loaded and processed:
  x_test shape: (10000, 784), dtype: float32
  y_test shape: (10000, 10), dtype: float64
Data check -> Min: 0.0, Max: 1.0
MNIST test data loaded and preprocessed.

Performing forward pass with unlocked weights...

Testing Accuracy (with unlocked weights): 5.34%


#### Perform predictions with locked model

In [ ]:
# --- Perform predictions ---
print("\nPerforming forward pass with unlocked weights...")
Y_pred = ft.forward_pass_mnist(x_test, w1, b1, w2, b2, w3, b3, w_out, b_out)

# Decode predictions to class labels
predicted_labels = np.argmax(Y_pred, axis=1)

# Compute and print testing accuracy
testing_accuracy = ft.calculate_accuracy(predicted_labels, y_test)

print(f"\nMNIST MLP Accuracy after unlock: {testing_accuracy:.2f}%")

#### Unlock MNIST MLP

In [ ]:
# Create a dictionary of weights to pass to the unlock function
locked_weights_dict = {
    'W1': w1,
    'W2': w2,
    'W3': w3,
    'W4': w_out # Map w_out to W4 for the unlock function
}

print("\nUnlocking weights...")
unlocked_weights = ft.unlock_weights(locked_weights_dict, unlock_key_processed)

# Update the global weight variables with the unlocked versions
w1 = unlocked_weights['W1']
w2 = unlocked_weights['W2']
w3 = unlocked_weights['W3']
w_out = unlocked_weights['W4'] # Update w_out with the unlocked W4


Unlocking weights...

--- Locked Weights Stats ---
w1 locked mean: -0.0049, std: 0.0693

--- Unlocked Weights Stats ---
w1 unlocked mean: -0.0049, std: 0.0693
Weights unlocked successfully!


#### Perform predictions with unlocked model

In [ ]:
# --- Perform predictions ---
print("\nPerforming forward pass with unlocked weights...")
Y_pred = ft.forward_pass_mnist(x_test, w1, b1, w2, b2, w3, b3, w_out, b_out)

# Decode predictions to class labels
predicted_labels = np.argmax(Y_pred, axis=1)

# Compute and print testing accuracy
testing_accuracy = ft.calculate_accuracy(predicted_labels, y_test)

binary_string = "".join(map(str, binarized_bits))
decimal_value = int(binary_string, 2)
response_after = hex(decimal_value)
print(f"\nResponse before: {response_before}")
print(f"Response after:  {response_after}")
print(f"\nMNIST MLP Accuracy after unlock: {testing_accuracy:.2f}%")